In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [25]:
design.get_chip_z('Q_chip')

'7um'

In [3]:
design.chips['C_chip'].size['size_x']='6mm'
design.chips['C_chip'].size['size_y']='6mm'
design.chips['Q_chip'].size['size_x']='4mm'
design.chips['Q_chip'].size['size_y']='4mm'
design.chips['Q_chip'].size['center_z']='7um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35+0.51-0.1+0.4-0.2
    
    resonator_length_4.append(a)
print(resonator_length_4)


[4.486627341935143, 4.372394169920711, 4.264173270104566]


# Flipchip

In [22]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='Q_chip',
                                            pad_width='355um',
                                            pad_height='80um',
                                            pad_gap='50um',
                                            pocket_width='650um',
                                            pocket_height='650um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+65}um",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [8]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))

In [9]:


stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [10]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [11]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','465um']


In [21]:
#3.91mm
rr0=connect('rr0', 'c0', 'in','stg', 'short', 
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1448um', '100um')

# EPR Analysis

In [23]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [24]:
hfss.start()

INFO 04:48PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:48PM [load_ansys_project]: 	Opened Ansys App
INFO 04:48PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:48PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   flip7um_V4_2
INFO 04:48PM [connect_design]: 	Opened active design
	Design:    flip7um_V4-10 [Solution type: Eigenmode]
INFO 04:48PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:48PM [connect]: 	Connected to project "flip7um_V4_2" and design "flip7um_V4-10" 😀 



True

In [25]:
hfss.activate_ansys_design("flip7um_V5", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

04:48PM 40s WARNING [activate_ansys_design]: The design_name=flip7um_V5 was not in active project.  Designs in active project are: 
['flip7um_V4-9', 'flip7um_V4-10'].  A new design will be added to the project.  
INFO 04:48PM [connect_design]: 	Opened active design
	Design:    flip7um_V5 [Solution type: Eigenmode]
WARNING 04:48PM [connect_setup]: 	No design setup detected.
WARNING 04:48PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:48PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [43]:
hfss.close()

True

# LOM Analysis

In [26]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [27]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.23',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.23',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [29]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp961jqpmh.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5230000000, Maxwell, 1, False
INFO 04:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwbsvxopk.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5230000000, Maxwell, 1, False
INFO 04:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpy797gg01.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5230000000, Maxwell, 2, False
INFO 04:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpn5rbn4_j.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5230000000, Maxwell, 3, False
INFO 04:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp2w05p6it.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5230000000, Maxwell, 4, False
INFO 04:59

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,317.200,-94.94700,-77.435,-81.87800
pad_bot_Q0,-94.947,110.74000,-14.844,-0.12031
pad_top_Q0,-77.435,-14.84400,110.760,-17.56900
quad_coupler_c0,-81.878,-0.12031,-17.569,99.76100


In [30]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.80
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.662920 [GHz]
EC 300.958354 [MHz]
EJ 14.854145 [GHz]
alpha -346.059301 [MHz]
dispersion 13.238628 [KHz]
Lq 10.995564 [nH]
Cq 64.361822 [fF]
T1 21.244387 [us]

**Coupling Properties**

tCqbus1 -8.727291 [fF]
gbus1_in_MHz -131.333012 [MHz]
χ_bus1 -7.157784 [MHz]
1/T1bus1 7491.623252 [Hz]
T1bus1 21.244387 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.080388,350.287753,14.854145,-408.602261,53.644493,[-138.26639623293073],[-19.340323244141885],19.340323,138.266396
2,5.779294,314.280222,14.854145,-362.792991,19.962924,[-147.6455537380849],[-11.303762790078409],11.303763,147.645554
3,5.691776,304.231009,14.854145,-350.159456,14.681341,[-141.08400965041196],[-8.716769286220696],8.716769,141.084010
4,5.676364,302.480589,14.854145,-347.965577,13.894099,[-137.17621228344476],[-8.006230996147968],8.006231,137.176212
5,5.665096,301.204411,14.854145,-346.367333,13.342781,[-135.19164736284358],[-7.615161851852517],7.615162,135.191647
6,5.655525,300.122849,14.854145,-345.013646,12.890054,[-133.10632422317553],[-7.252710234749242],7.252710,133.106324
7,5.660932,300.733668,14.854145,-345.778057,13.144118,[-131.9146959157625],[-7.194845873192348],7.194846,131.914696
8,5.66292,300.958354,14.854145,-346.059301,13.238628,[-131.3330115473295],[-7.157783569168562],7.157784,131.333012


In [25]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
